In [ ]:
Packages <- c("lmerTest", "arrow", "mgcv", "dplyr" , "effects")
invisible(lapply(Packages, library, character.only = TRUE))

In [ ]:
packageVersion("rlang")

In [ ]:
datafolder <- '/home/hoyinchan/blue/Data/data2022/'

In [ ]:
meds <- c('MEDATCA07DA',
 'MEDATCC03CA',
 'MEDATCC03CB',
 'MEDATCC03EB',
 'MEDATCC03AA',
 'MEDATCC03AB',
 'MEDATCC03AH',
 'MEDATCC03BA',
 'MEDATCC03EA',
 'MEDATCC09DX',
 'MEDATCC03DA',
 'MEDATCC03XA',
 'MEDATCC09XA',
 'MEDATCC03AX',
 'MEDATCC09BX',
 'MEDATCM05BA',
 'MEDATCM05BB')

In [ ]:
#loincs <- c('28233', '178616', '29512', '21600')
# lab_loinc = {'2823-3':'potassium',
#             '17861-6':'calcium',
#             '2951-2':'sodium', 
#             '2160-0':'sCr'}


loincs <- c('LG49936-4', 'LG49864-8', 'LG11363-5', 'LG6657-3')
# lab_loinc = {'LG49936-4':'potassium',
#                 'LG49864-8':'calcium',
#                 'LG11363-5':'sodium', 
#                 'LG6657-3':'sCr'}

loincs <- gsub("-", "", loincs)

In [ ]:
for (m in meds){
    for (l in loincs){
        tryCatch({
            path1 = paste0(datafolder,'xtitsa_',l,m,'.parquet')
            if (!file.exists(path1)) {
                cat(path1, "do not exists")
                next
            }
            
            df <- arrow::read_parquet(path1)
            df <- df %>% filter(DAYS_SINCE_ADMIT>=-7) %>% filter(DAYS_SINCE_ADMIT<=7)
            df$posttreatment <- as.factor(df$posttreatment)
#            x <- glm(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df)
            x <- lmer(RESULT_NUM ~  (1|ID) + (0 + DAYS_SINCE_ADMIT|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
                      control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))
            
            # x <- lmer(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
            #           control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))            
            print(paste(m, l, signif(fixef(x)[['posttreatment1']], digits = 2), coef(summary(x))['posttreatment1', 'Pr(>|t|)'], coef(summary(x))['posttreatment1', 'Pr(>|t|)']<0.05))
            print(paste(m, l, signif(fixef(x)[['posttreatment1:DAYS_SINCE_ADMIT']], digits = 2), coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)'], coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)']<0.05))
            flush.console()
            confx<-confint(x, method="Wald")
            df$predict <- predict(x, newdata = df, type = "response", re.form = NA)
            df$intercept <- fixef(x)[['posttreatment1']]
            df$interceptp <- coef(summary(x))['posttreatment1', 'Pr(>|t|)']
            df$slope <- fixef(x)[['posttreatment1:DAYS_SINCE_ADMIT']]
            df$slopep <- coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)']             
            df$interceptconfl <- confx['posttreatment1','2.5 %']
            df$interceptconfh <- confx['posttreatment1','97.5 %']
            df$slopeconfl <- confx['posttreatment1:DAYS_SINCE_ADMIT','2.5 %']
            df$slopeconfh <- confx['posttreatment1:DAYS_SINCE_ADMIT','97.5 %']            
            df$l <- l
            df$m <- m
#            path2 = paste0(datafolder,'xtitsa_result_',l,m,'.parquet')            
            path2 = paste0(datafolder,'xtitsa_result_norslope_',l,m,'.parquet')
            arrow::write_parquet(df, path2)
                },                 
#            print(paste(m, l, signif(fixef(x)[['posttreatment:DAYS_SINCE_ADMIT']], digits = 2)))},            
#        warning = function(w){},
        error = function(e) {print(paste(m, l, e))})
    }
}

In [ ]:
print('done')

In [ ]:
# l <- '21600'
# m <- 'MEDATCA07DA'

# path1 = paste0(datafolder,'xtitsa_',l,m,'.parquet')
# df <- arrow::read_parquet(path1)
# df <- df %>% filter(DAYS_SINCE_ADMIT>=-7) %>% filter(DAYS_SINCE_ADMIT<=7)
# df$posttreatment <- as.factor(df$posttreatment)
# #            x <- glm(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df)
# x <- lmer(RESULT_NUM ~  (1|ID) + (0 + DAYS_SINCE_ADMIT|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
#           control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))

In [ ]:
# x

In [ ]:
# coef(x)$ID

In [ ]:
# summary(x)

In [ ]:
# confint(x, method="Wald")